# VoiceMap App Opens — Data Consolidation

This notebook collects the three CSV exports of VoiceMap app open events, standardises the `Created at` date field, and prepares a master dataset for downstream reporting and mapping.

> Note: Run the cells locally to build the consolidated dataset; the CSV files are large, so keep an eye on memory usage.



In [1]:
from pathlib import Path

import pandas as pd

DATA_DIR = Path("/Users/mjsteenberg/11 Nov 25 - App Opens Data VoiceMap")
DATA_FILES = [
    DATA_DIR / "november2024-february2025.csv",
    DATA_DIR / "march2025-july2025.csv",
    DATA_DIR / "august2025-november2025.csv",
]

DATA_DIR, DATA_FILES


(PosixPath('/Users/mjsteenberg/11 Nov 25 - App Opens Data VoiceMap'),
 [PosixPath('/Users/mjsteenberg/11 Nov 25 - App Opens Data VoiceMap/november2024-february2025.csv'),
  PosixPath('/Users/mjsteenberg/11 Nov 25 - App Opens Data VoiceMap/march2025-july2025.csv'),
  PosixPath('/Users/mjsteenberg/11 Nov 25 - App Opens Data VoiceMap/august2025-november2025.csv')])

## Quick Glance at Each Batch

The snippet below reads only the first 50 rows from each CSV so you can sanity-check schema consistency before loading everything. Comment out when you no longer need the preview to reduce I/O overhead.


In [2]:
preview_frames = {}
for path in DATA_FILES:
    preview_frames[path.name] = pd.read_csv(path, nrows=50)

preview_frames


{'november2024-february2025.csv':     Id        Lat         Lng  Distance               Created at  \
 0    1  51.510400   -0.130100         0  November 21, 2024 07:37   
 1    2  51.510400   -0.130100         0  November 21, 2024 09:03   
 2    3  51.510400   -0.130100         0  November 21, 2024 09:04   
 3    4  51.510400   -0.130100         0  November 21, 2024 09:06   
 4    5  51.510400   -0.130100         0  November 22, 2024 03:58   
 5    6  51.510400   -0.130100         0  November 22, 2024 03:59   
 6    7  51.510400   -0.130100         0  November 22, 2024 04:02   
 7    8  51.510400   -0.130100         0  November 22, 2024 04:03   
 8    9  51.510400   -0.130100         0  November 22, 2024 04:06   
 9   10  51.510400   -0.130100         0  November 22, 2024 04:09   
 10  11  51.510400   -0.130100         0  November 22, 2024 04:13   
 11  12  51.510400   -0.130100         0  November 22, 2024 05:02   
 12  13  51.510400   -0.130100         0  November 22, 2024 05:05   
 

## Build Master Dataset

Load each CSV lazily, parse the date columns, standardise `Created at` to dates (midnight UTC), and append into a single `DataFrame` sorted by date.


In [3]:
DATE_COLS = ["Created at", "Updated at"]

frames = []
for path in DATA_FILES:
    df = pd.read_csv(path, parse_dates=DATE_COLS)
    df["Title [Route]"] = df["Title [Route]"].str.strip()
    df["Created at"] = df["Created at"].dt.normalize()
    frames.append(df)

master = pd.concat(frames, ignore_index=True)
master.sort_values("Created at", inplace=True)
master.reset_index(drop=True, inplace=True)

master.head()


,Id,Lat,Lng,Distance,Created at,Updated at,Title [Route],Id [User]
0,1,51.5104,-0.1301,0,2024-11-21,2024-11-21 07:37:00,Theatreland Tour with Ian McKellen,295130
1,2,51.5104,-0.1301,0,2024-11-21,2024-11-21 09:03:00,Theatreland Tour with Ian McKellen,295130
2,3,51.5104,-0.1301,0,2024-11-21,2024-11-21 09:04:00,Theatreland Tour with Ian McKellen,295130
3,4,51.5104,-0.1301,0,2024-11-21,2024-11-21 09:06:00,Theatreland Tour with Ian McKellen,295130
4,25,51.5104,-0.1301,0,2024-11-22,2024-11-22 09:39:00,Theatreland Tour with Ian McKellen,15325


In [4]:
summary = {
    "rows": len(master),
    "cols": master.columns.tolist(),
    "date_range": master["Created at"].agg(["min", "max"]).to_dict(),
    "null_counts": master.isna().sum().to_dict(),
}
summary


{'rows': 666903,
 'cols': ['Id',
  'Lat',
  'Lng',
  'Distance',
  'Created at',
  'Updated at',
  'Title [Route]',
  'Id [User]'],
 'date_range': {'min': Timestamp('2024-11-21 00:00:00'),
  'max': Timestamp('2025-11-11 00:00:00')},
 'null_counts': {'Id': 0,
  'Lat': 0,
  'Lng': 0,
  'Distance': 0,
  'Created at': 0,
  'Updated at': 0,
  'Title [Route]': 0,
  'Id [User]': 0}}

## Persist Output

Optionally write the consolidated dataset to disk for reuse in analytics notebooks or Streamlit apps.


In [5]:
OUTPUT_CSV = DATA_DIR / "voicemap_app_opens_master.csv"
OUTPUT_PARQUET = DATA_DIR / "voicemap_app_opens_master.parquet"

master.to_csv(OUTPUT_CSV, index=False)

try:
    master.to_parquet(OUTPUT_PARQUET, index=False)
except Exception as exc:  # pragma: no cover - parquet write is optional
    print(f"Skipping parquet export ({exc})")

OUTPUT_CSV, OUTPUT_PARQUET


(PosixPath('/Users/mjsteenberg/11 Nov 25 - App Opens Data VoiceMap/voicemap_app_opens_master.csv'),
 PosixPath('/Users/mjsteenberg/11 Nov 25 - App Opens Data VoiceMap/voicemap_app_opens_master.parquet'))

## Next: Streamlit Map Prototype

With the consolidated dataset in place, the next step is to build a Streamlit view that:

- Loads `voicemap_app_opens_master.parquet` (or the CSV fallback)
- Exposes a date range selector driven by `Created at`
- Filters points and visualises them on a map using `st.map` or `pydeck_chart`
- (Optional) adds tooltips with `Title [Route]`, `Distance`, and `Id [User]`

See the Streamlit skeleton below for a quick start.


```python
# streamlit_app.py
from pathlib import Path

import pandas as pd
import streamlit as st

DATA_PATH = Path("/Users/mjsteenberg/11 Nov 25 - App Opens Data VoiceMap/voicemap_app_opens_master.parquet")

@st.cache_data
def load_data() -> pd.DataFrame:
    if DATA_PATH.suffix == ".parquet":
        df = pd.read_parquet(DATA_PATH)
    else:
        df = pd.read_csv(DATA_PATH)
    df["Created at"] = pd.to_datetime(df["Created at"])
    return df

def main():
    df = load_data()

    min_date, max_date = df["Created at"].min(), df["Created at"].max()
    start_date, end_date = st.date_input(
        "Select date range",
        value=(min_date.date(), max_date.date()),
        min_value=min_date.date(),
        max_value=max_date.date(),
    )

    mask = (df["Created at"] >= pd.Timestamp(start_date)) & (df["Created at"] <= pd.Timestamp(end_date))
    filtered = df.loc[mask].copy()

    st.write(f"Showing {len(filtered):,} points")
    st.map(filtered[["Lat", "Lng"]].rename(columns={"Lat": "latitude", "Lng": "longitude"}))

    with st.expander("Raw data"):
        st.dataframe(filtered)

if __name__ == "__main__":
    main()
```
